In [19]:
#Import Libraries
import re
import pandas as pd

## Transform the Dataset

Firstly we import the dataset (ChessData.txt) and we transform it to a csv file

In [20]:
#Open the ChessData file
file = open("ChessData\chessData.txt", "r")

#Initialize empty lists for each attribute
White=[]
Black=[]
Date=[]
HalfMoves=[]
Moves=[]
Result=[]
WhiteElo=[]
BlackElo=[]
GameNumber=[]
Event=[]
Site=[]
EventDate=[]
Round=[]
ECO=[]
Opening=[]

l1=[]
l2=[]
l3=[]
l4=[]
l5=[]

#Take the values with regular expressions
for line in file:
    
    #FOR THE GAME SCV
    
    #For white attribute
    m1 = re.search("^White:\s([\D]+)", line)
    if m1 != None:
        White.append(m1.group(1).rstrip())

    #For black attribute
    m2 = re.search("^Black:\s([\D]+)", line)
    if m2 != None:
        Black.append(m2.group(1).rstrip())
    
    #For date attribute
    m3 = re.search("^Date:\s([0-9|.|?]+)", line)
    if m3 != None:
        Date.append(m3.group(1))
            
    #For Moves attribute
    m4 = re.search("^Moves:\s([\d]+)\s", line)
    if m4 != None:
        Moves.append(m4.group(1))
        
    #For Result attribute
    m5 = re.search("^Result:\s([\D]+)\s", line)
    if m5 != None:
        Result.append(m5.group(1))
        
    #For WhiteElo attribute
    m6 = re.search("^WhiteElo:\s([\d]+)\s", line)
    if m6 != None:
        WhiteElo.append(m6.group(1))
        
    #For BlackElo attribute
    m7 = re.search("^BlackElo:\s([\d]+)\s", line)
    if m7 != None:
        BlackElo.append(m7.group(1))
        
    #For GameNumber attribute
    m8 = re.search("^GameNumber:\s([\d]+)\s", line)
    if m8 != None:
        GameNumber.append(m8.group(1))
        
    #For Event attribute
    m9 = re.search("^Event:\s([\D\d]+)\\n", line)
    if m9 != None:
        Event.append(m9.group(1)) 
           
    #For EventDate attribute
    m10 = re.search("^EventDate:\s([0-9|.|?]+)", line)
    if m10 != None:
        EventDate.append(m10.group(1))
    
    #For Round attribute
    m11 = re.search("^Round:\s([\d]+)\s", line)
    if m11 != None:
        Round.append(m11.group(1))
        
    #For ECO attribute
    m12 = re.search("^ECO:\s([\D\d]+)\s", line)
    if m12 != None:
        ECO.append(m12.group(1))
           
    #For Opening attribute
    m13 = re.search("^Opening:\s([\D\d']+)", line)
    if m13 != None:
        Opening.append(m13.group(1).rstrip())
        
    #For Site attribute
    m14 = re.search("^Site:\s([\D]+)\s", line)
    if m14 != None:
        Site.append(m14.group(1)) 
        
    #For halfMoves attribute
    m15 = re.search("^HalfMoves:\s([\d]+)\s", line)
    if m15 != None:
        HalfMoves.append(m15.group(1))
        
    #FOR THE GAMEMOVES CSV
    
    m = re.search("^MoveNumber:\s(\d+),  Side: (\D+),  Move: ([\D\d']+),  FEN: ([\D\d]+),  GameNumber: (\d+)", line)
    if m != None:
        l1.append(m.group(1))
        l2.append(m.group(2))
        l3.append(m.group(3))
        l4.append(m.group(4))
        l5.append(m.group(5))


#A minor transformation in date in order to have the Neo4J format
date_new=[]
for date in Date:
    date_new.append(date.replace(".","-"))
Date=date_new  
        
#Create a dataframe
df= pd.DataFrame({'White': White,
                 'Black': Black,
                 'Date': Date,
                 'HalfMoves':HalfMoves,
                 'Moves': Moves,
                 'Result': Result,
                 'WhiteElo': WhiteElo,
                 'BlackElo': BlackElo,
                 'GameNumber': GameNumber,
                 'Event': Event,
                 'Site': Site,
                 'EventDate': EventDate,
                 'Round': Round,
                 'ECO': ECO,
                 'Opening': Opening})


#Create a dataframe
df2= pd.DataFrame({'MoveNumber': l1,
                 'Side': l2,
                 'Move': l3,
                 'FEN':l4,
                 'GameNumber': l5,
                 })


## Create the load CSV files

### Game Nodes

In [24]:
#Create Game nodes csv
df.to_csv('Game_Nodes.csv')

#Data preview
df.head(3)

,Black,BlackElo,Date,ECO,Event,EventDate,GameNumber,HalfMoves,Moves,Opening,Result,Round,Site,White,WhiteElo
0,Steinitz Wilhelm,0,1886-01-11,D11,World Championship 1st,1886.01.11,1,92,46,Queen's Gambit Declined Slav,Black,1,USA,Zukertort Johannes H,0
1,Zukertort Johannes H,0,1886-01-13,C47,World Championship 1st,1886.01.11,2,92,46,Four Knights,Black,2,USA,Steinitz Wilhelm,0
2,Steinitz Wilhelm,0,1886-01-15,D10,World Championship 1st,1886.01.11,3,93,46,Queen's Gambit Declined Slav,White,3,USA,Zukertort Johannes H,0


### Game-Position Edges

In [25]:
#Select only the first positions in order to create connections between games and first node
df3=df2.loc[df2.MoveNumber=='1']

#Drop columns that we don't need
df3.drop(['MoveNumber'], axis=1)

#Create Game Position Edge csv
df3.to_csv('Game_Position.csv')

#Data preview
df3.head(3)

,FEN,GameNumber,Move,MoveNumber,Side
0,rnbqkbnrpppppppp883P48PPP1PPPPRNBQKBNR,1,d4,1,white
92,rnbqkbnrpppppppp884P38PPPP1PPPRNBQKBNR,2,e4,1,white
184,rnbqkbnrpppppppp883P48PPP1PPPPRNBQKBNR,3,d4,1,white


### Position-Position Edges

In [26]:
#Remove the first positions in order to create connections oosition nodes

#Create two lists
FEN_start=list(df2.FEN)
FEN_end=list(df2.FEN)
move=list(df2.Move)
side=list(df2.Side)
game=list(df2.GameNumber)

#Drop the last element from the first list
del FEN_start[-1]

#Drop the first element of the second list
del FEN_end[0]

#Drop the last rows
del move[0]
del side[0]
del game[0]

connections=pd.DataFrame({'starting_position':FEN_start, 'ending_position':FEN_end, 'Move':move, 'Side':side, 'GameNumber':game})

#Create Node-Node edge
connections.to_csv('Move_edge.csv')

#Data preview
connections.GameNumber.nunique
connections.head(3)

,GameNumber,Move,Side,ending_position,starting_position
0,1,d5,black,rnbqkbnrppp1pppp83p43P48PPP1PPPPRNBQKBNR,rnbqkbnrpppppppp883P48PPP1PPPPRNBQKBNR
1,1,c4,white,rnbqkbnrppp1pppp83p42PP48PP2PPPPRNBQKBNR,rnbqkbnrppp1pppp83p43P48PPP1PPPPRNBQKBNR
2,1,c6,black,rnbqkbnrpp2pppp2p53p42PP48PP2PPPPRNBQKBNR,rnbqkbnrppp1pppp83p42PP48PP2PPPPRNBQKBNR


### Position Nodes

In [27]:
#Drop all the columns except FEN
df2.drop(['GameNumber', 'Move','MoveNumber','Side'], axis=1)

#Keep only unique values
positions=pd.DataFrame({'FEN':df2.FEN.unique()})

#Create Position nodes csv
positions.to_csv('Position_Nodes.csv')

#Data preview
positions.head(3)

,FEN
0,rnbqkbnrpppppppp883P48PPP1PPPPRNBQKBNR
1,rnbqkbnrppp1pppp83p43P48PPP1PPPPRNBQKBNR
2,rnbqkbnrppp1pppp83p42PP48PP2PPPPRNBQKBNR
